In [1]:
import pandas as pd
from path import Path
from sqlalchemy import create_engine
import psycopg2 as pg


In [2]:
# read postgres table, vaerscombined_2 into dataframe
connection = pg.connect(host='team-11.ch3wmigorcf0.us-east-2.rds.amazonaws.com', dbname="postgres", user="postgres", password='UTB00tcamp')
vaers_data_vax_df = pd.read_sql_query('select * from public."vaerscombined_2"',con=connection)
vaers_data_vax_df.head()


,VAERS_ID,AGE_YRS,SEX,DIED,L_THREAT,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_SITE,VAX_NAME
0,916602,23.0,F,0,None,None,None,None,None,Shellfish,COVID19,PFIZERBIONTECH,EL1284,1,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
1,916603,58.0,F,0,None,None,none,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","Diclofenac, novacaine, lidocaine, pickles, tom...",COVID19,MODERNA,unknown,UNK,None,COVID19 (COVID19 (MODERNA))
2,916606,44.0,F,0,None,None,None,None,None,iodine (shellfish) has epipen,COVID19,MODERNA,011J20A,1,LA,COVID19 (COVID19 (MODERNA))
3,916609,71.0,F,0,None,None,Medication Summary 1/1/21 Name of Medica...,none,"Hashimoto's thyroiditis, Hypertension, depression","Sulfa antibiotics, azithromycin, adhesive in ...",COVID19,MODERNA,011J201A,1,LA,COVID19 (COVID19 (MODERNA))
4,916617,35.0,F,0,None,None,"Fluoxetine 40mg qd, cetirizine 10mg qd Apple c...",None,Depression,None,COVID19,PFIZERBIONTECH,Eh9899,1,LA,COVID19 (COVID19 (PFIZER-BIONTECH))


In [3]:
len(vaers_data_vax_df.index)

76559

In [4]:
vaers_data_vax_df['VAX_TYPE'].value_counts()


COVID19       31179
VARZOS        11525
FLU4           6462
PPV            2582
FLUX           2072
UNK            1592
VARCEL         1543
PNC13          1521
HPV9           1462
TDAP           1340
HEPA           1121
FLUC4          1075
MMR            1043
MMRV           1043
MNQ             944
HIBV            909
RV5             905
FLUR4           837
HEP             826
TYP             803
MENB            715
FLUA4           691
DTAPIPVHIB      566
DTAPIPV         561
DTAP            559
FLU3            431
FLUA3           391
DTAPHEPBIP      359
IPV             303
HPV4            194
RV1             173
RAB             132
TD              118
ANTH             92
HEPAB            80
MEN              64
PNC              57
FLUN4            48
JEV1             44
TTOX             30
SMALL            30
YF               28
FLUN3            20
CHOL             19
DT               14
HPVX             12
FLUC3             9
DTP               9
FLUR3             6
RVX               5


In [5]:
# Drop VAX_TYPE not equal to COVID19
vaers_data_vax_df = vaers_data_vax_df[vaers_data_vax_df.VAX_TYPE == "COVID19"]

In [6]:
vaers_data_vax_df['VAX_TYPE'].value_counts()


COVID19    31179
Name: VAX_TYPE, dtype: int64

In [7]:
#Drop Unnecessary columns from VAERS_VAX_COVID19_df
vaers_data_vax_clean_df= vaers_data_vax_df.drop(["VAX_TYPE", "VAX_LOT", "VAX_SITE", "VAX_NAME"], axis=1).copy()
vaers_data_vax_clean_df.head()


,VAERS_ID,AGE_YRS,SEX,DIED,L_THREAT,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,VAX_MANU,VAX_DOSE_SERIES
0,916602,23.0,F,0,None,None,None,None,None,Shellfish,PFIZERBIONTECH,1
1,916603,58.0,F,0,None,None,none,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","Diclofenac, novacaine, lidocaine, pickles, tom...",MODERNA,UNK
2,916606,44.0,F,0,None,None,None,None,None,iodine (shellfish) has epipen,MODERNA,1
3,916609,71.0,F,0,None,None,Medication Summary 1/1/21 Name of Medica...,none,"Hashimoto's thyroiditis, Hypertension, depression","Sulfa antibiotics, azithromycin, adhesive in ...",MODERNA,1
4,916617,35.0,F,0,None,None,"Fluoxetine 40mg qd, cetirizine 10mg qd Apple c...",None,Depression,None,PFIZERBIONTECH,1


In [8]:
vaers_data_vax_clean_df.ALLERGIES.apply(lambda x: pd.Series(str(x).split(",")))

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,Shellfish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Diclofenac,novacaine,lidocaine,pickles,tomatoes,milk,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,iodine (shellfish) has epipen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sulfa antibiotics,azithromycin,adhesive in band-aids or tape,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76554,None; seasonal allergies,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76555,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76556,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76557,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
vaers_data_vax_clean_df['Allergy_1'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[0]
vaers_data_vax_clean_df['Allergy_2'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[1]
vaers_data_vax_clean_df['Allergy_3'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[2]
vaers_data_vax_clean_df['Allergy_4'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[3]
vaers_data_vax_clean_df['Allergy_5'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[4]
vaers_data_vax_clean_df['Allergy_6'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[5]
vaers_data_vax_clean_df['Allergy_7'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[6]
vaers_data_vax_clean_df['Allergy_8'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[7]
vaers_data_vax_clean_df['Allergy_9'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[8]
vaers_data_vax_clean_df['Allergy_10'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[9]
vaers_data_vax_clean_df['Allergy_11'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[10]
vaers_data_vax_clean_df['Allergy_12'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[11]
vaers_data_vax_clean_df['Allergy_13'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[12]
vaers_data_vax_clean_df['Allergy_14'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[13]
vaers_data_vax_clean_df['Allergy_15'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[14]
vaers_data_vax_clean_df['Allergy_16'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[15]
vaers_data_vax_clean_df['Allergy_17'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[16]
vaers_data_vax_clean_df['Allergy_18'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[17]
vaers_data_vax_clean_df['Allergy_19'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[18]
vaers_data_vax_clean_df['Allergy_20'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[19]
vaers_data_vax_clean_df['Allergy_21'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[20]
vaers_data_vax_clean_df['Allergy_22'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[21]
vaers_data_vax_clean_df['Allergy_23'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[22]
vaers_data_vax_clean_df['Allergy_24'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[23]
vaers_data_vax_clean_df['Allergy_25'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[24]
vaers_data_vax_clean_df['Allergy_26'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[25]
vaers_data_vax_clean_df['Allergy_27'] = vaers_data_vax_clean_df['ALLERGIES'].str.split(',', expand=True)[26]




In [10]:
vaers_data_vax_clean_df.head()

,VAERS_ID,AGE_YRS,SEX,DIED,L_THREAT,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,...,Allergy_18,Allergy_19,Allergy_20,Allergy_21,Allergy_22,Allergy_23,Allergy_24,Allergy_25,Allergy_26,Allergy_27
0,916602,23.0,F,0,None,None,None,None,None,Shellfish,...,None,None,None,None,None,None,None,None,None,None
1,916603,58.0,F,0,None,None,none,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","Diclofenac, novacaine, lidocaine, pickles, tom...",...,None,None,None,None,None,None,None,None,None,None
2,916606,44.0,F,0,None,None,None,None,None,iodine (shellfish) has epipen,...,None,None,None,None,None,None,None,None,None,None
3,916609,71.0,F,0,None,None,Medication Summary 1/1/21 Name of Medica...,none,"Hashimoto's thyroiditis, Hypertension, depression","Sulfa antibiotics, azithromycin, adhesive in ...",...,None,None,None,None,None,None,None,None,None,None
4,916617,35.0,F,0,None,None,"Fluoxetine 40mg qd, cetirizine 10mg qd Apple c...",None,Depression,None,...,None,None,None,None,None,None,None,None,None,None


In [11]:
vaers_data_vax_clean_df.count(0)


VAERS_ID           31179
AGE_YRS            28760
SEX                31179
DIED               31179
L_THREAT            1099
HOSPITAL            3549
OTHER_MEDS         19294
CUR_ILL            15628
HISTORY            20786
ALLERGIES          18322
VAX_MANU           31179
VAX_DOSE_SERIES    31162
Allergy_1          18322
Allergy_2           3829
Allergy_3           2241
Allergy_4           1309
Allergy_5            801
Allergy_6            497
Allergy_7            335
Allergy_8            218
Allergy_9            143
Allergy_10           107
Allergy_11            79
Allergy_12            56
Allergy_13            42
Allergy_14            36
Allergy_15            27
Allergy_16            24
Allergy_17            19
Allergy_18            16
Allergy_19            12
Allergy_20            11
Allergy_21             8
Allergy_22             7
Allergy_23             5
Allergy_24             5
Allergy_25             2
Allergy_26             2
Allergy_27             1
dtype: int64

In [12]:
vaers_data_vax_clean_df = vaers_data_vax_clean_df.fillna('')

In [13]:
vaers_data_vax_clean_df

,VAERS_ID,AGE_YRS,SEX,DIED,L_THREAT,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,...,Allergy_18,Allergy_19,Allergy_20,Allergy_21,Allergy_22,Allergy_23,Allergy_24,Allergy_25,Allergy_26,Allergy_27
0,916602,23.0,F,0,,,None,None,None,Shellfish,...,,,,,,,,,,
1,916603,58.0,F,0,,,none,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","Diclofenac, novacaine, lidocaine, pickles, tom...",...,,,,,,,,,,
2,916606,44.0,F,0,,,,,,iodine (shellfish) has epipen,...,,,,,,,,,,
3,916609,71.0,F,0,,,Medication Summary 1/1/21 Name of Medica...,none,"Hashimoto's thyroiditis, Hypertension, depression","Sulfa antibiotics, azithromycin, adhesive in ...",...,,,,,,,,,,
4,916617,35.0,F,0,,,"Fluoxetine 40mg qd, cetirizine 10mg qd Apple c...",,Depression,None,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76554,916586,51.0,F,0,,,Vitamin D TriaMeereen-hydrochlorothiazide,None,High blood pressure,None; seasonal allergies,...,,,,,,,,,,
76555,916594,91.0,M,0,,,Statin ASA Metformin Lisinopril Anoro Celexia ...,none,DM HTN RENAL FAILURE 2,none,...,,,,,,,,,,
76556,918433,,U,0,,,,,,,...,,,,,,,,,,
76557,918437,,F,0,,,,,,,...,,,,,,,,,,


In [14]:
vaers_allergy_df= vaers_data_vax_clean_df.drop(['ALLERGIES','AGE_YRS','SEX','DIED','L_THREAT','HOSPITAL','OTHER_MEDS','CUR_ILL','HISTORY','VAX_MANU','VAX_DOSE_SERIES'], axis=1).copy()
                                                

In [15]:
vaers_allergy_df['Allergy_1'] = vaers_allergy_df['Allergy_1'].replace(['N/a','None.','None','none','NKDA or food allergies','nka','NKA','No','no','Unknown','unknown','None known',
                                                                       'none known','no known allergies','No known allergies','No known allergies.','UKN','No, none','NONE LISTED',
                                                                       'no known','NKDA/NKA','none that i know of','Knda'],"")



In [17]:
vaers_allergy_df.head()


,VAERS_ID,Allergy_1,Allergy_2,Allergy_3,Allergy_4,Allergy_5,Allergy_6,Allergy_7,Allergy_8,Allergy_9,...,Allergy_18,Allergy_19,Allergy_20,Allergy_21,Allergy_22,Allergy_23,Allergy_24,Allergy_25,Allergy_26,Allergy_27
0,916602,Shellfish,,,,,,,,,...,,,,,,,,,,
1,916603,Diclofenac,novacaine,lidocaine,pickles,tomatoes,milk,,,,...,,,,,,,,,,
2,916606,iodine (shellfish) has epipen,,,,,,,,,...,,,,,,,,,,
3,916609,Sulfa antibiotics,azithromycin,adhesive in band-aids or tape,,,,,,,...,,,,,,,,,,
4,916617,,,,,,,,,,...,,,,,,,,,,
